CIFAR-10

https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py

# Import modules

In [1]:
import copy
from datetime import datetime
import gc
import logging
from logging import getLogger
from logging import StreamHandler
from logging import INFO, WARNING
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from tensorboardX import SummaryWriter
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import warnings
warnings.filterwarnings("ignore")

logger = getLogger(__name__)
handler = StreamHandler()
handler.setLevel(INFO)
logger.setLevel(INFO)
logger.addHandler(handler)

%load_ext jupyternotify

<IPython.core.display.Javascript object>

# Global vars

In [2]:
TRAIN = "train"
VAL = "val"
TEST = "test"
PHASES = [TRAIN, VAL]

SAVE_PATH = os.path.join(os.getcwd(), "models")

# Utils

In [3]:
def display_formatted_time(elapsed_time, msg=""):
    minutes, seconds = map(int, divmod(elapsed_time, 60))
    print("Elapsed time - {0}: {1}min {2}s".format(msg, minutes, seconds))

In [4]:
def save_model(model, model_name=""):
    model_path = os.path.join(SAVE_PATH, model_name)
    torch.save(model.state_dict(), model_path)

def load_model(model_name):
    model = Net().to(device)
    model_path = os.path.join(SAVE_PATH, model_name)
    model.load_state_dict(torch.load(model_path))
    return model

In [5]:
def generate_log_path():
    # return os.path.join("logs", "train_{}".format(datetime.utcnow().strftime("%Y%m%d%H%M%S")))
    return os.path.join("logs", "train")

# Settings

In [6]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Device: ", device)

torch.manual_seed(1)

test_batch_size = 256

# default
params = {
    "batch_size": 512,
    "epochs": 10,
    "lr": 0.001,
    "momentum": 0.9,
}

Device:  cuda


# Load image

In [7]:
def init_datasets(data_transforms):
    """画像前処理変更したら呼ぶ"""
    image_datasets = {phase: torchvision.datasets.CIFAR10(root="../../data",
                                                                                                       train=phase is "train",
                                                                                                       download=True,
                                                                                                       transform=data_transforms[phase])
                                      for phase in PHASES}
    dataset_sizes = {phase: len(image_datasets[phase]) for phase in PHASES}
    return image_datasets, dataset_sizes

def init_dataloaders(batch_size, image_datasets):
    """バッチサイズ変更したら呼ぶ"""
    return {phase: torch.utils.data.DataLoader(image_datasets[phase],
                                                                               batch_size=batch_size,
                                                                               shuffle=True,
                                                                               num_workers=4)
                 for phase in PHASES}

In [8]:
data_transforms = {
    "train": transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    "val": transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
}

image_datasets, dataset_sizes = init_datasets(data_transforms)
dataloaders = init_dataloaders(params["batch_size"], image_datasets)

classes = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck"
)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(dataloaders[TRAIN])
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print(" ".join("%5s" % classes[labels[j]] for j in range(params["batch_size"])))

# Define a Convolution Neural Network

In [ ]:
# from net import Net

In [28]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net,  self).__init__()
        self.conv1 = nn.Conv2d(3,  16,  5)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2,  2)
        self.conv2 = nn.Conv2d(16,  16,  5)
        self.bn2 = nn.BatchNorm2d(16)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(16 * 5 * 5,  120)
        self.fc2 = nn.Linear(120,  84)
        self.fc3 = nn.Linear(84,  10)

    def forward(self,  x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # 32x32x3 ->  28x28x6 ->14x14x6
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  #  ->10x10x16 -> 5x5x16
        x = x.view(-1,  16 * 5 * 5)  # -> 400
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)
        return x

# Train the network

In [9]:
def train(model, criterion, optimizer, scheduler, params):
    since = time.time()
    
    # Tensorboard
    log_path = generate_log_path()
    writer = SummaryWriter(log_path)
    
    epochs = params["epochs"]

    best_model_weights = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(epochs):
        epoch_loss = dict()
        epoch_acc = dict()
        
        for phase in PHASES:
            if phase == TRAIN:
                is_train = True
                scheduler.step()
                model.train()
            else:
                is_train = False
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            
            for batch_idx, (inputs, labels) in enumerate(dataloaders[phase], 0):
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(is_train):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if is_train:
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss[phase] = running_loss / dataset_sizes[phase]
            epoch_acc[phase] = running_corrects.double() / dataset_sizes[phase]

            if phase == VAL and epoch_acc[phase] > best_acc:
                best_acc = epoch_acc[phase]
                best_model_weights = copy.deepcopy(model.state_dict())

        print("Epoch {}/{}\tLoss Train: {:.4f} Val: {:.4f}\tAcc Train: {:.4f} Val: {:.4f}".format(
            epoch,
            epochs - 1,
            epoch_loss[TRAIN],
            epoch_loss[VAL],
            epoch_acc[TRAIN],
            epoch_acc[VAL]
        ))
        writer.add_scalars(
            "losses",
            {
                "train_loss": epoch_loss[TRAIN],
                "val_loss": epoch_loss[VAL],
            },
            epoch
        )
        writer.add_scalars(
            "acc",
            {
                "train_acc": epoch_acc[TRAIN],
                "val_acc": epoch_acc[VAL]
            },
            epoch
        )

    print()
    print("Best val Acc: {:4f}".format(best_acc))
    
    writer.close()
    model.load_state_dict(best_model_weights)
    gc.collect()
    
    print()
    display_formatted_time(time.time() - since)
    return model

In [10]:
def calc_acc(model):
    """ラベルごとの精度を算出"""
    class_correct = [0. for i in range(len(classes))]
    class_total = [0. for i in range(len(classes))]

    with torch.no_grad():
        for inputs, labels in dataloaders[VAL]:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(10):
        print("Accuracy of\t%5s:\t%2d %%" % (classes[i], 100 * class_correct[i] / class_total[i]))

In [11]:
def generate_optimizer(model, params, algo="Adam"):
    if algo == "Adam":
        return optim.Adam(model.parameters(),
                                          lr=params["lr"],
                                          weight_decay=params["weight_decay"])
    elif algo =="SGD":
        return optim.SGD(model.parameters(),
                                        lr=params["lr"],
                                        momentum=params["momentum"])
    else:
        return

In [17]:
def generate_scheduler(optimizer, method="cos"):
    if method == "cos":
        return lr_scheduler.CosineAnnealingLR(optimizer=optimizer,
                                                                            T_max=10,  # Maximum number of iterations
                                                                            eta_min=0,  # 最小学習率
                                                                            last_epoch=-1)  # The index of last epoch
    elif method == "step":
        return  lr_scheduler.StepLR(optimizer,
                                                         step_size=7,
                                                         gamma=0.1)
    else:
        return

## Train and Eval

In [13]:
gc.collect()

7

In [42]:
params = {
    "batch_size": 512,
    "epochs": 50,
    "lr": 0.001,
    "momentum": 0.9,
    "weight_decay": 1e-5,
}

image_datasets, dataset_sizes = init_datasets(data_transforms=data_transforms)
dataloaders = init_dataloaders(params["batch_size"], image_datasets=image_datasets)
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = generate_optimizer(model, params, "Adam")
scheduler = generate_scheduler(optimizer, "cos")

model = train(model, criterion, optimizer, scheduler, params)

Files already downloaded and verified
Files already downloaded and verified
Epoch 0/49	Loss Train: 1.8130 Val: 1.4782	Acc Train: 0.3172 Val: 0.4590
Epoch 1/49	Loss Train: 1.4296 Val: 1.2875	Acc Train: 0.4757 Val: 0.5315
Epoch 2/49	Loss Train: 1.2963 Val: 1.2037	Acc Train: 0.5360 Val: 0.5609
Epoch 3/49	Loss Train: 1.2263 Val: 1.1444	Acc Train: 0.5629 Val: 0.5878
Epoch 4/49	Loss Train: 1.1743 Val: 1.1139	Acc Train: 0.5833 Val: 0.5991
Epoch 5/49	Loss Train: 1.1375 Val: 1.0737	Acc Train: 0.5992 Val: 0.6146
Epoch 6/49	Loss Train: 1.1051 Val: 1.0566	Acc Train: 0.6108 Val: 0.6182
Epoch 7/49	Loss Train: 1.0812 Val: 1.0391	Acc Train: 0.6178 Val: 0.6276
Epoch 8/49	Loss Train: 1.0739 Val: 1.0275	Acc Train: 0.6224 Val: 0.6334
Epoch 9/49	Loss Train: 1.0685 Val: 1.0253	Acc Train: 0.6260 Val: 0.6329
Epoch 10/49	Loss Train: 1.0662 Val: 1.0252	Acc Train: 0.6254 Val: 0.6332
Epoch 11/49	Loss Train: 1.0635 Val: 1.0244	Acc Train: 0.6259 Val: 0.6337
Epoch 12/49	Loss Train: 1.0677 Val: 1.0227	Acc Train: 0.62

In [45]:
calc_acc(model)

Accuracy of	plane:	77 %
Accuracy of	  car:	85 %
Accuracy of	 bird:	59 %
Accuracy of	  cat:	52 %
Accuracy of	 deer:	70 %
Accuracy of	  dog:	62 %
Accuracy of	 frog:	82 %
Accuracy of	horse:	79 %
Accuracy of	 ship:	83 %
Accuracy of	truck:	80 %


## Save model

In [65]:
save_model(model, "original_Adam_epoch50")

## Load model

In [10]:
model = load_model("original_SGD_epoch50")

# Learning rate annealing

Cosine Annealing

In [41]:
params = {
    "batch_size": 2048,
    "epochs": 100,
    "lr": 0.001,
    "momentum": 0.9,
}

image_datasets, dataset_sizes = init_datasets(data_transforms=data_transforms)
dataloaders = init_dataloaders(params["batch_size"], image_datasets=image_datasets)
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                                            lr=params["lr"])
scheduler = lr_scheduler.CosineAnnealingLR(optimizer=optimizer,
                                                                              T_max=10,  # Maximum number of iterations
                                                                              eta_min=0,  # 最小学習率
                                                                              last_epoch=-1)  # The index of last epoch

model = train(model, criterion, optimizer, scheduler, params)

Files already downloaded and verified
Files already downloaded and verified
Epoch 0/199	Train Loss: 2.0250 Acc: 0.2619	Val Loss: 1.9228 Acc: 0.3014
Epoch 1/199	Train Loss: 1.5971 Acc: 0.4066	Val Loss: 1.6279 Acc: 0.3907
Epoch 2/199	Train Loss: 1.4276 Acc: 0.4755	Val Loss: 1.4042 Acc: 0.4819
Epoch 3/199	Train Loss: 1.3346 Acc: 0.5152	Val Loss: 1.3171 Acc: 0.5187
Epoch 4/199	Train Loss: 1.2696 Acc: 0.5433	Val Loss: 1.2726 Acc: 0.5378
Epoch 5/199	Train Loss: 1.2269 Acc: 0.5576	Val Loss: 1.2141 Acc: 0.5627
Epoch 6/199	Train Loss: 1.1959 Acc: 0.5707	Val Loss: 1.1913 Acc: 0.5759
Epoch 7/199	Train Loss: 1.1735 Acc: 0.5813	Val Loss: 1.1807 Acc: 0.5779
Epoch 8/199	Train Loss: 1.1618 Acc: 0.5844	Val Loss: 1.1698 Acc: 0.5807
Epoch 9/199	Train Loss: 1.1559 Acc: 0.5865	Val Loss: 1.1662 Acc: 0.5832
Epoch 10/199	Train Loss: 1.1524 Acc: 0.5882	Val Loss: 1.1660 Acc: 0.5830
Epoch 11/199	Train Loss: 1.1520 Acc: 0.5882	Val Loss: 1.1654 Acc: 0.5850
Epoch 12/199	Train Loss: 1.1524 Acc: 0.5869	Val Loss: 1.16

Epoch 112/199	Train Loss: 0.6452 Acc: 0.7760	Val Loss: 0.8016 Acc: 0.7294
Epoch 113/199	Train Loss: 0.6494 Acc: 0.7736	Val Loss: 0.8164 Acc: 0.7198
Epoch 114/199	Train Loss: 0.6541 Acc: 0.7709	Val Loss: 0.8124 Acc: 0.7236
Epoch 115/199	Train Loss: 0.6550 Acc: 0.7699	Val Loss: 0.8397 Acc: 0.7161
Epoch 116/199	Train Loss: 0.6577 Acc: 0.7692	Val Loss: 0.8330 Acc: 0.7158
Epoch 117/199	Train Loss: 0.6655 Acc: 0.7663	Val Loss: 0.8356 Acc: 0.7187
Epoch 118/199	Train Loss: 0.6699 Acc: 0.7632	Val Loss: 0.8465 Acc: 0.7112
Epoch 119/199	Train Loss: 0.6590 Acc: 0.7688	Val Loss: 0.8449 Acc: 0.7124
Epoch 120/199	Train Loss: 0.6633 Acc: 0.7665	Val Loss: 0.8473 Acc: 0.7124
Epoch 121/199	Train Loss: 0.6553 Acc: 0.7705	Val Loss: 0.8400 Acc: 0.7151
Epoch 122/199	Train Loss: 0.6529 Acc: 0.7703	Val Loss: 0.9113 Acc: 0.6965
Epoch 123/199	Train Loss: 0.6512 Acc: 0.7718	Val Loss: 0.8278 Acc: 0.7196
Epoch 124/199	Train Loss: 0.6376 Acc: 0.7764	Val Loss: 0.8242 Acc: 0.7203
Epoch 125/199	Train Loss: 0.6275 Acc: 

In [42]:
calc_acc(model)

Accuracy of	plane:	77 %
Accuracy of	  car:	84 %
Accuracy of	 bird:	63 %
Accuracy of	  cat:	53 %
Accuracy of	 deer:	71 %
Accuracy of	  dog:	62 %
Accuracy of	 frog:	82 %
Accuracy of	horse:	79 %
Accuracy of	 ship:	82 %
Accuracy of	truck:	81 %


In [43]:
save_model("original_Adam_CosineAnnealing_batchsize2048_epoch200")

StepLR

In [ ]:
params = {
    "batch_size": 1024,
    "epochs": 50,
    "lr": 0.001,
    "momentum": 0.9,
}

dataloaders = init_dataloaders(batch_size)
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                                            lr=params["lr"])
scheduler = lr_scheduler.StepLR(optimizer=optimizer,
                                                     step_size=10,
                                                     gamma=0.9)

model = train(model, criterion, optimizer, scheduler, params)

In [ ]:
calc_acc(model)

# Transfer Learning

## Finetuning ResNet-18

In [46]:
data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    "val": transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
}

image_datasets, dataset_sizes = init_datasets(data_transforms)
dataloaders = init_dataloaders(params["batch_size"], image_datasets)

Files already downloaded and verified
Files already downloaded and verified


In [58]:
del dataloaders; gc.collect()

0

In [59]:
params = {
    "batch_size": 32,
    "epochs": 10,
    "lr": 0.001,
    "momentum": 0.9,
    "weight_decay": 1e-5,
}

data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    "val": transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
}

image_datasets, dataset_sizes = init_datasets(data_transforms)
dataloaders = init_dataloaders(params["batch_size"], image_datasets)

# Load pretrained model
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.conv1 = nn.Conv2d(in_channels=3,
                                                    out_channels=64,
                                                    kernel_size=7,
                                                    stride=2,
                                                    padding=3,
                                                    bias=False)
model_ft.fc = nn.Linear(num_ftrs, 10)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = generate_optimizer(model_ft, params, "Adam")
scheduler = generate_scheduler(optimizer, "cos")

model_ft = train(model_ft, criterion, optimizer, scheduler, params)

Files already downloaded and verified
Files already downloaded and verified
Epoch 0/9	Loss Train: 1.5141 Val: 1.2037	Acc Train: 0.4587 Val: 0.5722
Epoch 1/9	Loss Train: 1.1095 Val: 1.0641	Acc Train: 0.6092 Val: 0.6336
Epoch 2/9	Loss Train: 0.9595 Val: 0.9358	Acc Train: 0.6643 Val: 0.6711
Epoch 3/9	Loss Train: 0.8530 Val: 0.8154	Acc Train: 0.7047 Val: 0.7114
Epoch 4/9	Loss Train: 0.7749 Val: 0.7330	Acc Train: 0.7300 Val: 0.7451
Epoch 5/9	Loss Train: 0.6925 Val: 0.6436	Acc Train: 0.7598 Val: 0.7752
Epoch 6/9	Loss Train: 0.6300 Val: 0.6113	Acc Train: 0.7813 Val: 0.7872
Epoch 7/9	Loss Train: 0.5681 Val: 0.5421	Acc Train: 0.8039 Val: 0.8082
Epoch 8/9	Loss Train: 0.5192 Val: 0.5410	Acc Train: 0.8194 Val: 0.8152
Epoch 9/9	Loss Train: 0.4948 Val: 0.5309	Acc Train: 0.8277 Val: 0.8172

Best val Acc: 0.817200

Elapsed time - : 25min 23s


In [63]:
calc_acc(model_ft)

Accuracy of	plane:	82 %
Accuracy of	  car:	87 %
Accuracy of	 bird:	77 %
Accuracy of	  cat:	71 %
Accuracy of	 deer:	81 %
Accuracy of	  dog:	80 %
Accuracy of	 frog:	87 %
Accuracy of	horse:	82 %
Accuracy of	 ship:	86 %
Accuracy of	truck:	87 %


In [66]:
save_model(model_ft, "resnet18_Adam_CosineAnnealing_batchsize32_epoch10")

In [67]:
gc.collect()

859

### Fixed feature extractor

In [19]:
params = {
    "batch_size": 128,
    "epochs": 15,
    "lr": 0.001,
    "momentum": 0.9,
    "weight_decay": 1e-5,
}

data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    "val": transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
}

image_datasets, dataset_sizes = init_datasets(data_transforms)
dataloaders = init_dataloaders(params["batch_size"], image_datasets)

# Load pretrained model
model_ft = models.resnet18(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad = False
num_ftrs = model_ft.fc.in_features
model_ft.conv1 = nn.Conv2d(in_channels=3,
                                                    out_channels=64,
                                                    kernel_size=7,
                                                    stride=2,
                                                    padding=3,
                                                    bias=False)
model_ft.fc = nn.Linear(num_ftrs, 10)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = generate_optimizer(model_ft, params, "Adam")
scheduler = generate_scheduler(optimizer, "step")

model_ft = train(model_ft, criterion, optimizer, scheduler, params)

Files already downloaded and verified
Files already downloaded and verified
Epoch 0/14	Loss Train: 1.7771 Val: 1.5793	Acc Train: 0.3616 Val: 0.4458
Epoch 1/14	Loss Train: 1.5004 Val: 1.4649	Acc Train: 0.4706 Val: 0.4807
Epoch 2/14	Loss Train: 1.4187 Val: 1.3718	Acc Train: 0.5001 Val: 0.5164
Epoch 3/14	Loss Train: 1.3660 Val: 1.3702	Acc Train: 0.5180 Val: 0.5216
Epoch 4/14	Loss Train: 1.3098 Val: 1.3126	Acc Train: 0.5412 Val: 0.5424
Epoch 5/14	Loss Train: 1.2748 Val: 1.2948	Acc Train: 0.5519 Val: 0.5417
Epoch 6/14	Loss Train: 1.2252 Val: 1.1718	Acc Train: 0.5700 Val: 0.5858
Epoch 7/14	Loss Train: 1.1644 Val: 1.1419	Acc Train: 0.5926 Val: 0.5955
Epoch 8/14	Loss Train: 1.1502 Val: 1.1211	Acc Train: 0.6002 Val: 0.5999
Epoch 9/14	Loss Train: 1.1430 Val: 1.1217	Acc Train: 0.6021 Val: 0.6100
Epoch 10/14	Loss Train: 1.1350 Val: 1.1273	Acc Train: 0.6051 Val: 0.6119
Epoch 11/14	Loss Train: 1.1264 Val: 1.1102	Acc Train: 0.6079 Val: 0.6108
Epoch 12/14	Loss Train: 1.1245 Val: 1.1028	Acc Train: 0.60

In [ ]:
save_model(model_ft, "resnet18_freeze_Adam_StepLR_batchsize128_epoch15_lr1e-2")

## Finetuning VGG

In [71]:
params = {
    "batch_size": 32,
    "epochs": 10,
    "lr": 0.001,
    "momentum": 0.9,
    "weight_decay": 1e-5,
}

data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    "val": transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
}

image_datasets, dataset_sizes = init_datasets(data_transforms)
dataloaders = init_dataloaders(params["batch_size"], image_datasets)

# Load pretrained model
model_ft = models.vgg16(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad = False
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs, 10)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = generate_optimizer(model_ft, params, "Adam")
scheduler = generate_scheduler(optimizer, "cos")

model_ft = train(model_ft, criterion, optimizer, scheduler, params)

Files already downloaded and verified
Files already downloaded and verified
Epoch 0/9	Loss Train: 1.3554 Val: 1.1196	Acc Train: 0.5135 Val: 0.6026
Epoch 1/9	Loss Train: 1.3158 Val: 1.1093	Acc Train: 0.5382 Val: 0.6000
Epoch 2/9	Loss Train: 1.3008 Val: 1.1036	Acc Train: 0.5443 Val: 0.6041
Epoch 3/9	Loss Train: 1.2982 Val: 1.0775	Acc Train: 0.5450 Val: 0.6162
Epoch 4/9	Loss Train: 1.2704 Val: 1.0930	Acc Train: 0.5524 Val: 0.6141
Epoch 5/9	Loss Train: 1.2499 Val: 1.0476	Acc Train: 0.5594 Val: 0.6311
Epoch 6/9	Loss Train: 1.2265 Val: 1.0747	Acc Train: 0.5628 Val: 0.6173
Epoch 7/9	Loss Train: 1.2105 Val: 1.0547	Acc Train: 0.5689 Val: 0.6312
Epoch 8/9	Loss Train: 1.1885 Val: 1.0487	Acc Train: 0.5788 Val: 0.6353
Epoch 9/9	Loss Train: 1.1955 Val: 1.0453	Acc Train: 0.5727 Val: 0.6346

Best val Acc: 0.635300

Elapsed time - : 44min 48s


In [73]:
save_model(model_ft, "vgg16_freeze_Adam_CosineAnnealing_batchsize32_epoch10")

In [ ]:
since = time.time()

for epoch in range(epochs):

    running_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(dataloader[TRAIN], 0):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        exp_lr_scheduler.step()

        running_loss += loss.item()
        if batch_idx % log_interval == (log_interval - 1):
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tRunningLoss: {:.3f}".format(
                epoch, batch_idx * len(inputs), len(dataloader[TRAIN].dataset),
                100. * batch_idx / len(dataloader[TRAIN]), loss.item(), running_loss / log_interval
            ))
            running_loss = 0.0

display_formatted_time(time.time() - since)

In [ ]:
since = time.time()

correct = 0
total = 0
with torch.no_grad():
    for (inputs, labels) in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_ft(inputs)
        _, predicted = torch.max(outputs, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print("Accuracy of the network on the 10000 test images: %d %%" % (100 * correct / total))

display_formatted_time(time.time() - since)

In [ ]:
print(labels)
print(outputs)